In [150]:
import pandas as pd
import datetime
import math

pd.options.mode.chained_assignment = None 


# move the location of the excel sheet
excel_file = 'SIP.xlsx' #stored within the same Jupyter folder
#excel_file = '/Users/jbanerjee/Desktop/SIP2.xlsx' #if the excel sheet is stored at any other location

#read 2 sheets from the excel file - Main and Interest
sip = pd.read_excel(excel_file, sheet_name = ['Main','Interest'])

#change them to list so that it can be converted to dataframe
table1 = sip['Main'].values.tolist()
table2 = sip['Interest'].values.tolist()

#change to dataframe
df = pd.DataFrame(table1[1:], columns=table1[0])
dfI = pd.DataFrame(table2[1:10], columns=table2[0])

dfI = dfI.rename(columns={'Year': 'Date-Year', 'Rate': 'Interest Rate' })


#select only those 2 columns that is required to do the job
df = df.iloc[:, [0,7]]

date_time_str = '2000-01-01 00:00:00.000000'
date_time_obj = datetime.datetime.strptime(date_time_str, '%Y-%m-%d %H:%M:%S.%f')


#Change Date value to 2000-01-01 if it is str or float or int or bool or complex or bytes datatype
df['Date'] = [date_time_obj if (type(x) == str or type(x) == float or type(x) == int or type(x) == bool or type(x) == complex or type(x) == bytes) else x for x in df['Date']]


#Change Invested value to 0 if it is str or bool or complex or bytes datatype
df['Invested in market from Salary'] = [0 if (type(x) == str or type(x) == bool or type(x) == complex or type(x) == bytes) else x for x in df['Invested in market from Salary']]


#Change Date to Qtr
df['Date-Qtr'] = df['Date'].dt.to_period("Q")


#get all unique Qtr values
a = df['Date-Qtr'].unique()


#initialise list b and z
b = [] #will have the year-quarter like 2017Q3
z = [] #will have the amount of money invested in that quarter of that year


for i in range (len(a)):
    
    #increase the length of the list b and z by 1
    b = b + ['']
    z = z + ['']
    
    #concatenate the unique date with the word sum. This becomes my new variable
    b[i] = str(a[i])+"sum"
    
    #create a dynamic variable name same as Qtr of the year and store the sum in it
    name = b[i]
    
    #Sum all values of that Qtr and store it in a variable
    value = df.loc[df['Date-Qtr'] == a[i], 'Invested in market from Salary'].sum()
    dict = {name: value}
    
    #create variable z that will contain total amount invested for that quarter
    z[i] = dict[b[i]]
    
#create a new dataframe with the new data
dfn = pd.DataFrame({"Date-Qtr": a, "Quarter":b, "Total Amount Invested":z})

#modify the new dataframe to have Year at the beginning and define a new column Interest1 with default value 0.
#Interest1 will later be modified to capture the values of rate of interest from the excel sheet. See dfI dataframe for more
dfn = pd.DataFrame({"Date-Year": dfn['Date-Qtr'].dt.year, "Date-Qtr": a, "Quarter":b, "Total Amount Invested":z, "Interest1":float(0)})


for i in range (len(dfn['Date-Year'])):
    for j in range (len(dfI['Date-Year'])):
        if (dfn['Date-Year'][i] == dfI['Date-Year'][j]):
            dfn['Interest1'][i] = dfI['Interest Rate'][j] 
           

#Calculate number of quarters until now since investment and create a new column Number-of-Qtrs in the dataframe dfn
def function1(i):
    col = str(i)
    now = datetime.datetime.now()
    qtrvar = math.ceil(now.month/3.)
    yearvar = math.ceil(now.year)
    qtrdiff = (yearvar - a.year)*4 + abs((qtrvar - a.quarter))
    test = 5
    return qtrdiff


#Creating new column Number-of-Qtrs through the function1
dfn['Number-of-Qtrs'] = dfn['Date-Qtr'].apply(function1)

#Creating new column Amount = (1+ ((int rate)/4)/100) to the power Number-of-Qtrs * Principal amount
dfn['Amount'] = ((1+dfn['Interest1']/400)**dfn['Number-of-Qtrs'])*dfn['Total Amount Invested']

#Changing amount to int for readability
dfn['Amount'] = dfn['Amount'].astype(int) 

#Summing up all the Amounts to get the overall number
currentbalance = dfn['Amount'].sum()

#Sort the year-quarter values in descending order for readability purpose
dfn = dfn.sort_values(by=['Quarter'])


## Export dataframe to excel without deleting current sheets of excel
import pandas as pd
import numpy as np
from openpyxl import load_workbook

path = 'SIP.xlsx'

book = load_workbook(path)
writer = pd.ExcelWriter(path, engine = 'openpyxl') #version 3.0.4
writer.book = book

df3 = pd.DataFrame(dfn)
df3.to_excel(writer, sheet_name = 'Python')

print (currentbalance,"\n",dfn)
#The following 2 commands are commented, uncomment it when required to create the sheets on excel 
#writer.save()
#writer.close()




7404288 
     Date-Year Date-Qtr    Quarter  Total Amount Invested  Interest1  \
0        2015   2015Q1  2015Q1sum                   0.00       8.20   
1        2015   2015Q2  2015Q2sum                   0.00       8.20   
2        2015   2015Q3  2015Q3sum                   0.00       8.20   
3        2015   2015Q4  2015Q4sum                   0.00       8.20   
4        2016   2016Q1  2016Q1sum               26000.00       8.20   
5        2016   2016Q2  2016Q2sum              168000.00       8.20   
6        2016   2016Q3  2016Q3sum              535000.00       8.20   
7        2016   2016Q4  2016Q4sum              742000.00       8.20   
8        2017   2017Q1  2017Q1sum             1060000.00       7.75   
9        2017   2017Q2  2017Q2sum              735000.00       7.75   
10       2017   2017Q3  2017Q3sum              457400.00       7.75   
11       2017   2017Q4  2017Q4sum              667000.00       7.75   
12       2018   2018Q1  2018Q1sum              358183.02       7.50

In [135]:
import pandas as pd
dfa = dfn
#print (dfa)
#print (dfI)

dfa['Interest1'] = float(0)
print (dfa)

for i in range (len(dfa['Date-Year'])):
    for j in range (len(dfI['Date-Year'])):
        if (dfa['Date-Year'][i] == dfI['Date-Year'][j]):
            dfa['Interest1'][i] = dfI['Interest Rate'][j] 
           
print (dfa)



    Date-Year Date-Qtr    Quarter  Total Amount Invested  Interest Rate  \
0        2015   2015Q1  2015Q1sum                   0.00          10.62   
1        2015   2015Q2  2015Q2sum                   0.00          10.62   
2        2015   2015Q3  2015Q3sum                   0.00          10.62   
3        2015   2015Q4  2015Q4sum                   0.00          10.62   
4        2016   2016Q1  2016Q1sum               26000.00           8.20   
5        2016   2016Q2  2016Q2sum              168000.00           8.20   
6        2016   2016Q3  2016Q3sum              535000.00           8.20   
7        2016   2016Q4  2016Q4sum              742000.00           8.20   
8        2017   2017Q1  2017Q1sum             1060000.00           7.75   
9        2017   2017Q2  2017Q2sum              735000.00           7.75   
10       2017   2017Q3  2017Q3sum              457400.00           7.75   
11       2017   2017Q4  2017Q4sum              667000.00           7.75   
12       2018   2018Q1  2

<ipython-input-135-edacb138705d>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfa['Interest1'][i] = dfI['Interest Rate'][j]
